
A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [30]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [31]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [32]:
# Explore Object volumns and their value
for c in ds_jobs.select_dtypes('object').columns:
    print (c,'\n',ds_jobs[c].value_counts(),'\n')
    

city 
 city_103    4355
city_21     2702
city_16     1533
city_114    1336
city_160     845
            ... 
city_129       3
city_111       3
city_121       3
city_140       1
city_171       1
Name: city, Length: 123, dtype: int64 

gender 
 Male      13221
Female     1238
Other       191
Name: gender, dtype: int64 

relevant_experience 
 Has relevant experience    13792
No relevant experience      5366
Name: relevant_experience, dtype: int64 

enrolled_university 
 no_enrollment       13817
Full time course     3757
Part time course     1198
Name: enrolled_university, dtype: int64 

education_level 
 Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: education_level, dtype: int64 

major_discipline 
 STEM               14492
Humanities           669
Other                381
Business Degree      327
Arts                 253
No Major             223
Name: major_discipline, dtype: int64 

experience 
 >20    3286

In [ ]:
# dictionary for Categories with only 2 factors update to bool
two_values_categories = {
    'relevant_experience': {'Has relevant experience': True, 'No relevant experience': False},
    'job_change': {1.0: True ,0.0: False}
}

# Make Columns containing ordinal categorical data an order category
order_categories ={
    'enrolled_university': ['no_enrollment','Part time course','Full time course'],
    'education_level': ['Prmary School', 'High School','Graduate','Masters','Phd'],
    'experience': ['<1'] + list(map(str,range(1,21))) + ['>20'],
    'company_size': ['<10', '10_49','50-99','100-499','500-999','1000-4999','5000-9999','10000+'],
    'last_new_job': ['never','1','2','3','4','>4']
}


In [34]:
# Change the data types to the professional types
for c in ds_jobs_transformed:

    # Columns containing categories with only two factors must be stored as Booleans
    if c in ['relevant_experience','job_change']:
        ds_jobs_transformed[c] = ds_jobs_transformed[c].map(two_values_categories[c])
        
    # Columns containing integers only must be stored as 32-bit integers
    elif c in ['student_id','training_hours']:
        ds_jobs_transformed[c] = ds_jobs_transformed[c].astype('int32')
        
    # Columns containing floats must be stored as 16-bit floats
    elif c == 'city_development_index':
        ds_jobs_transformed[c] = ds_jobs_transformed[c].astype('float16')
        
    # Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to               numerical values, with an order that reflects the natural order of the column
    elif c in order_categories.keys():
        category = pd.CategoricalDtype(order_categories[c], ordered=True)
        ds_jobs_transformed[c] = ds_jobs_transformed[c].astype(category)
        
    # Columns containing nominal categorical data must be stored as the category
    else:
        ds_jobs_transformed[c] = ds_jobs_transformed[c].astype('category')
        

In [35]:
# filtered to only contain students with >=10 of experience at companies with at least 1000 employees
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['experience'] >= '10' ) & (ds_jobs_transformed['company_size'] >= '1000-4999')]

print(ds_jobs_transformed)

       student_id      city  ...  training_hours job_change
9             699  city_103  ...             123      False
12          25619   city_61  ...              23      False
31          22293  city_103  ...             141      False
34          26494   city_16  ...             145      False
40           2547  city_114  ...              14      False
...           ...       ...  ...             ...        ...
19097       25447  city_103  ...              57      False
19101        6803   city_16  ...              89      False
19103       32932   city_10  ...              18      False
19128        3365   city_16  ...              23      False
19143       33047  city_103  ...              18      False

[2201 rows x 14 columns]


In [36]:
# Explore our data after  changement
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   bool    
 5   enrolled_university     2185 non-null   category
 6   education_level         2177 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            2184 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   bool    
dtypes: bool(2), category(9)

In [37]:
# check menory usage for original data set
ds_jobs.memory_usage()

Index                        128
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64

In [38]:
# check menory usage for original data set
ds_jobs_transformed.memory_usage()

Index                     17608
student_id                 8804
city                       7353
city_development_index     4402
gender                     2333
relevant_experience        2201
enrolled_university        2333
education_level            2413
major_discipline           2421
experience                 2933
company_size               2565
company_type               2421
last_new_job               2421
training_hours             8804
job_change                 2201
dtype: int64